# EnigmaNet: A Neural Network Framework for Eplipesy Classification
### This is a script that uses Keras to predict epilepsy outcomes

## Script Details
This script is designed to run with the following configuration:
* Keras
* TensorFlow backend
* PlaidML backend (if using Radeon GPU)
* Tabular data (Excel or CSVs)

## Dependencies
Run the following command to pip install all dependencies
```
pip install numpy pandas sklearn matplotlib scipy keras_tqdm imblearn
```

Conda users can search for same packages and install them via `conda install <package_name>`

Also install NeuroCombat at https://github.com/ncullen93/neuroCombat

In [1]:
# Local modules
import enigma as en
from enigmalibrary import enigmamodels 

# Standard modules
import numpy as np

# Sklearn modules
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

# Keras modules
from keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


## Create Enigma Data Object
The Enigma data object holds the data, and enables the application of various transformations.

In [2]:
inData = en.enigmanet(path='/Users/sid/Documents/Projects/Enigma-ML/Dataset/T1/all.csv',
                 classcol='Dx',
                 sitecol='Site',
                 dbegin='ICV',
                 dend='R_insula_surfavg',
                 fillmissing=True,
                 harmonize=True,
                 crange=['Site', 'Sex'],
                 batch='Site',
                 discrete=['Dx', 'Sex'],
                 continuous=['Age'],
                 scale=True,
                 testsplit=0.30)

Found classes: [1 0]


In [3]:
inData.transformdata()

...found 2 classes across 16 sites
...filling missing data with class means
Creating design matrix..
Standardizing data across features..
Fitting L/S model and finding priors..
Finding parametric adjustments..
Final adjustment of data..
...scaling data


In [4]:
x_train, x_test, y_train, y_test = inData.splitdata(oversample=True)
print('Dataset containing ' + str(len(inData.dframe.loc[:, inData.classcol])) + ' observations split into:')
print('    Training: ' + str(len(y_train)))
print('    Testing: ' + str(len(y_test)))

...splitting data
...oversampling data
Dataset containing 2776 observations split into:
    Training: 2130
    Testing: 833


## Create GridSearch Dict


In [ ]:
hyperparameters = {'input_dim': [x_train.shape[1]],
                   'optimizer': ['Adam', 'Nadam', 'SGD', 'RMSprop'],
                   'activation': ['relu', 'sigmoid', 'linear', 'tanh', 'softmax'],
                   'initmode': ['glorot_uniform', 'glorot_normal', 'lecun_uniform', 'uniform'],
                   'dropout': [0.1, 0.2, 0.3],
                   'units_L1': [10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 500],
                   'units_L2': [2, 5, 10, 15, 20, 50, 100],
                   'L2_reg_penalty': [0.01, 0.015, 0.1, 0.15, 0.2],
                   'epochs': [50, 75, 100, 150, 200, 300, 500, 1000, 2000]}

enigmaClassifier = KerasClassifier(enigmamodels.CreateSequentialModel, batch_size=32, verbose=1)

grid = GridSearchCV(estimator=enigmaClassifier, param_grid=hyperparameters)
grid_result = grid.fit(x_train, y_train)


print('The parameters of the best model are: ')
print(grid_result.best_params_)

/Users/sid/miniconda3/envs/deep/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
W0925 14:46:09.049275 4592989632 deprecation_wrapper.py:119] From /Users/sid/miniconda3/envs/deep/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0925 14:46:09.066893 4592989632 deprecation_wrapper.py:119] From /Users/sid/miniconda3/envs/deep/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0925 14:46:09.069067 4592989632 deprecation_wrapper.py:119] From /Users/sid/miniconda3/envs/deep/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Pleas

Epoch 1/25
1420/1420 [==============================] - 0s 216us/step - loss: 7.6111 - acc: 0.5296
Epoch 2/25
1420/1420 [==============================] - 0s 32us/step - loss: 7.6059 - acc: 0.5296
Epoch 3/25
1420/1420 [==============================] - 0s 31us/step - loss: 7.6016 - acc: 0.5296
Epoch 4/25
1420/1420 [==============================] - 0s 27us/step - loss: 7.5981 - acc: 0.5296
Epoch 5/25
1420/1420 [==============================] - 0s 28us/step - loss: 7.5951 - acc: 0.5296
Epoch 6/25
1420/1420 [==============================] - 0s 28us/step - loss: 7.5927 - acc: 0.5296
Epoch 7/25
1420/1420 [==============================] - 0s 27us/step - loss: 7.5906 - acc: 0.5296
Epoch 8/25
1420/1420 [==============================] - 0s 27us/step - loss: 7.5890 - acc: 0.5296
Epoch 9/25
1420/1420 [==============================] - 0s 28us/step - loss: 7.5876 - acc: 0.5296
Epoch 10/25
1420/1420 [==============================] - 0s 27us/step - loss: 7.5865 - acc: 0.5296
Epoch 11/25
1420/1

1420/1420 [==============================] - 0s 29us/step - loss: 1.7553 - acc: 0.5944
Epoch 8/25
1420/1420 [==============================] - 0s 29us/step - loss: 1.5279 - acc: 0.6099
Epoch 9/25
1420/1420 [==============================] - 0s 29us/step - loss: 1.5899 - acc: 0.6169
Epoch 10/25
1420/1420 [==============================] - 0s 29us/step - loss: 1.3625 - acc: 0.6042
Epoch 11/25
1420/1420 [==============================] - 0s 29us/step - loss: 1.4100 - acc: 0.6134
Epoch 12/25
1420/1420 [==============================] - 0s 29us/step - loss: 1.3697 - acc: 0.6021
Epoch 13/25
1420/1420 [==============================] - 0s 30us/step - loss: 1.2568 - acc: 0.6324
Epoch 14/25
1420/1420 [==============================] - 0s 29us/step - loss: 1.0037 - acc: 0.6500
Epoch 15/25
1420/1420 [==============================] - 0s 28us/step - loss: 1.0040 - acc: 0.6486
Epoch 16/25
1420/1420 [==============================] - 0s 29us/step - loss: 0.8676 - acc: 0.6563
Epoch 17/25
1420/1420 [=

1420/1420 [==============================] - 0s 31us/step - loss: 0.7772 - acc: 0.6528
Epoch 14/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.7733 - acc: 0.6514
Epoch 15/25
1420/1420 [==============================] - 0s 31us/step - loss: 0.7896 - acc: 0.6563
Epoch 16/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.7687 - acc: 0.6620
Epoch 17/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.8370 - acc: 0.6268
Epoch 18/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.8482 - acc: 0.6493
Epoch 19/25
1420/1420 [==============================] - 0s 29us/step - loss: 0.7938 - acc: 0.6535
Epoch 20/25
1420/1420 [==============================] - 0s 29us/step - loss: 0.8026 - acc: 0.6507
Epoch 21/25
1420/1420 [==============================] - 0s 29us/step - loss: 0.7144 - acc: 0.6627
Epoch 22/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.7433 - acc: 0.6655
Epoch 23/25
1420/1420 

1420/1420 [==============================] - 0s 32us/step - loss: 1.1220 - acc: 0.6373
Epoch 20/25
1420/1420 [==============================] - 0s 31us/step - loss: 0.9985 - acc: 0.6486
Epoch 21/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.9038 - acc: 0.6415
Epoch 22/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.8468 - acc: 0.6711
Epoch 23/25
1420/1420 [==============================] - 0s 31us/step - loss: 0.8228 - acc: 0.6768
Epoch 24/25
1420/1420 [==============================] - 0s 30us/step - loss: 0.9262 - acc: 0.6655
Epoch 25/25
710/710 [==============================] - 0s 267us/step
Epoch 1/25
1420/1420 [==============================] - 1s 470us/step - loss: 5.6266 - acc: 0.5225
Epoch 2/25
1420/1420 [==============================] - 0s 36us/step - loss: 4.3844 - acc: 0.5169
Epoch 3/25
1420/1420 [==============================] - 0s 34us/step - loss: 2.6913 - acc: 0.5577
Epoch 4/25
1420/1420 [==============================] 

...

1420/1420 [==============================] - 0s 45us/step - loss: 2.9372 - acc: 0.5338
Epoch 4/25
1420/1420 [==============================] - 0s 42us/step - loss: 2.5792 - acc: 0.5535
Epoch 5/25
1420/1420 [==============================] - 0s 41us/step - loss: 2.5652 - acc: 0.5634
Epoch 6/25
1420/1420 [==============================] - 0s 41us/step - loss: 2.1980 - acc: 0.5838
Epoch 7/25
1420/1420 [==============================] - 0s 41us/step - loss: 2.2498 - acc: 0.5373
Epoch 8/25
1420/1420 [==============================] - 0s 42us/step - loss: 2.1221 - acc: 0.5211
Epoch 9/25
1420/1420 [==============================] - 0s 41us/step - loss: 1.9290 - acc: 0.5352
Epoch 10/25
1420/1420 [==============================] - 0s 41us/step - loss: 1.7609 - acc: 0.5549
Epoch 11/25
1420/1420 [==============================] - 0s 41us/step - loss: 1.5830 - acc: 0.6127
Epoch 12/25
1420/1420 [==============================] - 0s 41us/step - loss: 1.7049 - acc: 0.6000
Epoch 13/25
1420/1420 [=====

1420/1420 [==============================] - 0s 53us/step - loss: 8.9801 - acc: 0.4430
Epoch 10/25
1420/1420 [==============================] - 0s 50us/step - loss: 8.9795 - acc: 0.4430
Epoch 11/25
1420/1420 [==============================] - 0s 54us/step - loss: 8.9791 - acc: 0.4430
Epoch 12/25
1420/1420 [==============================] - 0s 48us/step - loss: 8.9789 - acc: 0.4430
Epoch 13/25
1420/1420 [==============================] - 0s 44us/step - loss: 8.9787 - acc: 0.4430
Epoch 14/25
1420/1420 [==============================] - 0s 42us/step - loss: 8.9786 - acc: 0.4430
Epoch 15/25
1420/1420 [==============================] - 0s 42us/step - loss: 8.9786 - acc: 0.4430
Epoch 16/25
1420/1420 [==============================] - 0s 42us/step - loss: 8.9785 - acc: 0.4430
Epoch 17/25
1420/1420 [==============================] - 0s 42us/step - loss: 8.9785 - acc: 0.4430
Epoch 18/25
1420/1420 [==============================] - 0s 42us/step - loss: 8.9785 - acc: 0.4430
Epoch 19/25
1420/1420 

1420/1420 [==============================] - 0s 44us/step - loss: 0.8546 - acc: 0.6085
Epoch 16/25
1420/1420 [==============================] - 0s 47us/step - loss: 0.8570 - acc: 0.6225
Epoch 17/25
1420/1420 [==============================] - 0s 50us/step - loss: 0.8708 - acc: 0.6169
Epoch 18/25
1420/1420 [==============================] - 0s 47us/step - loss: 0.8056 - acc: 0.6148
Epoch 19/25
1420/1420 [==============================] - 0s 49us/step - loss: 0.8522 - acc: 0.6204
Epoch 20/25
1420/1420 [==============================] - 0s 46us/step - loss: 0.8800 - acc: 0.6310
Epoch 21/25
1420/1420 [==============================] - 0s 45us/step - loss: 0.7397 - acc: 0.6261
Epoch 22/25
1420/1420 [==============================] - 0s 43us/step - loss: 0.7736 - acc: 0.6451
Epoch 23/25
1420/1420 [==============================] - 0s 43us/step - loss: 0.7869 - acc: 0.6352
Epoch 24/25
1420/1420 [==============================] - 0s 44us/step - loss: 0.7616 - acc: 0.6423
Epoch 25/25
710/710 [=

1420/1420 [==============================] - 0s 44us/step - loss: 1.1460 - acc: 0.6472
Epoch 22/25
1420/1420 [==============================] - 0s 44us/step - loss: 1.0511 - acc: 0.6577
Epoch 23/25
1420/1420 [==============================] - 0s 44us/step - loss: 0.9668 - acc: 0.6570
Epoch 24/25
1420/1420 [==============================] - 0s 44us/step - loss: 1.0142 - acc: 0.6620
Epoch 25/25
710/710 [==============================] - 1s 1ms/step
Epoch 1/25
1420/1420 [==============================] - 2s 1ms/step - loss: 3.8496 - acc: 0.5430
Epoch 2/25
1420/1420 [==============================] - 0s 49us/step - loss: 3.4403 - acc: 0.5232
Epoch 3/25
1420/1420 [==============================] - 0s 47us/step - loss: 3.6842 - acc: 0.5648
Epoch 4/25
1420/1420 [==============================] - 0s 45us/step - loss: 3.3296 - acc: 0.5775
Epoch 5/25
1420/1420 [==============================] - 0s 45us/step - loss: 2.9170 - acc: 0.5845
Epoch 6/25
1420/1420 [==============================] - 0s 4

Epoch 2/25
1420/1420 [==============================] - 0s 55us/step - loss: 2.1776 - acc: 0.5986
Epoch 3/25
1420/1420 [==============================] - 0s 54us/step - loss: 1.9741 - acc: 0.6120
Epoch 4/25
1420/1420 [==============================] - 0s 49us/step - loss: 1.7143 - acc: 0.6162
Epoch 5/25
1420/1420 [==============================] - 0s 50us/step - loss: 1.5296 - acc: 0.6303
Epoch 6/25
1420/1420 [==============================] - 0s 50us/step - loss: 1.3427 - acc: 0.6732
Epoch 7/25
1420/1420 [==============================] - 0s 48us/step - loss: 1.2065 - acc: 0.6711
Epoch 8/25
1420/1420 [==============================] - 0s 48us/step - loss: 1.1017 - acc: 0.6810
Epoch 9/25
1420/1420 [==============================] - 0s 50us/step - loss: 1.0434 - acc: 0.7092
Epoch 10/25
1420/1420 [==============================] - 0s 49us/step - loss: 1.0328 - acc: 0.7127
Epoch 11/25
1420/1420 [==============================] - 0s 49us/step - loss: 0.9195 - acc: 0.7063
Epoch 12/25
1420/1